In [29]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import ShuffleSplit, KFold
import matplotlib.pyplot as plt
from scipy import sparse
from itertools import product
from time import time
import seaborn
%run Helpers.ipynb

In [41]:
n_components = [10, 20, 40, 60]
data = [U, Uhat]
num_user = X.shape[0]
metrics = ['cosine', 'minkowski']
n_neighbors = [5, 10, 20, 40]
#also should iterate over number of neighbors used

scores = np.zeros((2, 2, 4, 4))
times = np.zeros((2, 2, 4, 4))

user_test = Utest[:,-1].toarray().flatten().astype('int')
user_test_idx = np.arange(0, Utest.shape[0])

In [42]:
for i, j, k, h in product(range(len(data)), range(len(metrics)), range(len(n_components)), range(len(n_neighbors))):
    temp_score = []
    
    #make pca
    pca_data = TruncatedSVD(n_components=n_components[k]).fit_transform(data[i])
    
    #make and train neighbors
    nn  = NearestNeighbors(metric=metrics[j], algorithm='brute').fit(pca_data)
    start = time()
    #get nearest neighbors, making sure to exclude self
    #SHOULD I EXCLUDE SELF OR NOT?
    neighbors = nn.kneighbors(pca_data[user_test], n_neighbors[h])[1][1:]
    times[i,j,k,h] = time() - start
    
    #check if we got the recipe they liked
    for u, n in zip(user_test_idx, neighbors):
        temp_score.append( user_score_temp(u, n) )

    scores[i,j,k,h] = np.array(temp_score).mean()
    print(i,j,k,h, scores[i,j,k,h], times[i,j,k,h])
    
np.save("results/scores_userNN.npy", scores)
np.save("results/times_userNN.npy", times)

0 0 0 0 8.39360610060126 5.207545042037964
0 0 0 1 8.492770200909224 5.211828947067261
0 0 0 2 8.603284939140638 5.069382667541504
0 0 0 3 8.719929608446986 5.014098405838013
0 0 1 0 8.41283179351811 5.165575265884399
0 0 1 1 8.512113213081097 5.356341361999512
0 0 1 2 8.649948672825928 5.3854241371154785
0 0 1 3 8.77350051327174 5.188766241073608
0 0 2 0 8.443848071564746 5.0783531665802
0 0 2 1 8.534271887373515 5.04546046257019
0 0 2 2 8.632834726499487 5.085838794708252
0 0 2 3 8.758249010118785 5.05039119720459
0 0 3 0 8.428376594808622 5.012374639511108
0 0 3 1 8.513975656254583 4.993877410888672
0 0 3 2 8.601789118639097 5.064584493637085
0 0 3 3 8.723317201935767 5.4592673778533936
0 1 0 0 8.376990761108667 5.21156120300293
0 1 0 1 8.471638070098253 5.204097032546997
0 1 0 2 8.563938993987389 5.220836639404297
0 1 0 3 8.675890893092827 5.266160011291504
0 1 1 0 8.387578823874469 5.277464389801025
0 1 1 1 8.476052207068484 5.240615367889404
0 1 1 2 8.600293298137556 5.2276172637